# UTS Deep Learning Number 1
Name: Austin Kane\
NIM: 270222932\
Dataset: [Dataset B](https://drive.google.com/drive/folders/1pThuel6dU23CLRseKzymT4oNDUECq_c6)

# 1. Artificial Neural Network
Anda adalah seorang Data Scientist di perusahaan komersil. Tugas kalian adalah membuat model **Artificial Neural Network (ANN)** yang dapat menilai produktivitas setiap team dari perusahaan produsen pakaian.
Download dataset yang telah disediakan dan kerjakan task berikut ini.
1. [LO2 – 5 Poin] Lakukan **Exploratory Data Analysis (EDA)** untuk memahami kondisi data. Jelaskan semua masalah yang anda temukan pada dataset anda. Lakukan pre-processing pada dataset anda sesuai dengan hasil EDA anda, termasuk memisahkan dataset anda menjadi train, val, dan test dengan proporsi 70:10:20.
2. [LO2, LO3, & LO4 – 10 Poin] Buatlah **2 baseline model** dengan jumlah layer dan neuron yang berbeda. 1 model berupa **Sequential Model** dan 1 lagi berupa **Functional Model**. Semua hidden layer wajib menggunakan activation function bernama ReLU dan memiliki jumlah neuron minimal 2 kali lipat dari dimensi input data. Kedua model harus memiliki minimal 2 hidden layer. Lakukan training pada kedua model tersebut dengan minimal 10 epoch.
3. [LO2, LO3, & LO4 – 10 Poin] Lakukan **modifikasi pada kedua model anda**. Anda dapat mengubah jumlah neuron dan layer ataupun activation function dari hidden layer. Anda juga dapat melakukan **hyperparameter fine-tuning** pada model anda. Lakukan training pada 2 modifikasi model anda.
4. [LO2, LO3, & LO4 – 10 Poin] Lakukan **evaluasi pada 4 model** yang sudah anda buat menggunakan **minimal 3 evaluation metrics**, lalu bandingkan, analisis, dan simpulkan hasilnya.
5. [LO1, LO2 – 5 Poin] Buatlah **video presentasi** yang menjelaskan langkah-langkah pengerjaan anda serta hasil analisis anda dengan durasi maksimal 15 menit.




## Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Read the Data

In [3]:
pd.read_csv()

TypeError: read_csv() missing 1 required positional argument: 'filepath_or_buffer'

## Explaratory Data Analysis (EDA)

## Data Preprocessing

## Baseline Model

## Hyperparameter Fine-Tuning

## Evaluation